In [ ]:
import requests
import pandas as pd
import numpy as np
import json

## API used
https://rapidapi.com/movie-of-the-night-movie-of-the-night-default/api/streaming-availability/

In [ ]:
def request(page):
    url = "https://streaming-availability.p.rapidapi.com/search/basic"

    headers = {
        'x-rapidapi-host': "streaming-availability.p.rapidapi.com",
        'x-rapidapi-key': "e758c7a22amsh8ba3a4bd2a799b8p18abd9jsn7e529bfbb7a6"
        }
    querystring = {"country":"us","service":"hbo","type":"movie", "page":page}
    html = requests.get(url, headers=headers, params=querystring)
    single_movie_data = html.json()
    with open('single_movie_data.json', 'w', encoding='utf-8') as f:
        json.dump(single_movie_data['results'], f, ensure_ascii=False, indent=4)
    df = pd.read_json('single_movie_data.json', orient='records')
    return df

In [ ]:
def new_dataset():
    columns_names = ["imdbID", "tmdbID", "imdbRating",	"imdbVoteCount",	"tmdbRating",	"originalTitle", "genres",	"countries",	"year",	"runtime",	"cast",	"significants",	"title",	"overview",	"tagline",	"video",	"posterPath",	"poster_URLs_original",	"age",	"netflix",	"disney",	"prime",	"hbo"]
    Dataset = pd.DataFrame(columns = columns_names)
    return Dataset

In [ ]:
def write(df, Dataset):
    for i in range(8):
        md = df.iloc[i]
        try: 
            Dataset.loc[-1] = [md['imdbID'], md['tmdbID'], md['imdbRating'], md['imdbVoteCount'], md['tmdbRating'], md['originalTitle'], md['genres'], md['countries'], md['year'], md['runtime'], md['cast'], md['significants'], md['title'], md['overview'], md['tagline'], md['video'], md['posterPath'], md['posterURLs']['original'], md['age'], 'False', 'False', 'False', 'True']
        except: 
            Dataset.loc[-1] = [md['imdbID'], md['tmdbID'], md['imdbRating'], md['imdbVoteCount'], md['tmdbRating'], md['originalTitle'], md['genres'], md['countries'], md['year'], md['runtime'], md['cast'], md['significants'], md['title'], md['overview'], md['tagline'], md['video'], md['posterPath'], np.nan, md['age'], 'False', 'False', 'False', 'True']
        Dataset.index = Dataset.index + 1
        Dataset = Dataset.sort_index()
    return Dataset

In [ ]:
def main_function(start_page, page_count, Dataset):
    if start_page==0:
        dataset = new_dataset()
    else:
        dataset = Dataset
    for i in range(page_count):
        df = request(start_page + i + 1)
        dataset = write(df, dataset)
    return dataset

## Create Dataset

In [ ]:
dataset = main_function(0, 258, Dataset)

In [ ]:
dataset

In [ ]:
dataset.to_csv('Dataset.csv')

## Small tests

In [6]:
# url = "https://streaming-availability.p.rapidapi.com/get/basic"

# querystring = {"country":"us","imdb_id":"tt0264464","output_language":"en"}

# headers = {
#     'x-rapidapi-host': "streaming-availability.p.rapidapi.com",
#     'x-rapidapi-key': "e758c7a22amsh8ba3a4bd2a799b8p18abd9jsn7e529bfbb7a6"
#     }

# response = requests.request("GET", url, headers=headers, params=querystring)

# print(response.text)

{"message":"You are not subscribed to this API."}


In [ ]:
# url = "https://streaming-availability.p.rapidapi.com/search/basic"

# headers = {
#     'x-rapidapi-host': "streaming-availability.p.rapidapi.com",
#     'x-rapidapi-key': "e758c7a22amsh8ba3a4bd2a799b8p18abd9jsn7e529bfbb7a6"
#     }
# querystring = {"country":"us","service":"hbo","type":"movie"}
# html = requests.get(url, headers=headers, params=querystring)
# print(html.text)

{"results":[{"imdbID":"tt6959552","tmdbID":"510192","imdbRating":57,"imdbVoteCount":10,"tmdbRating":0,"backdropPath":"/xKmTsbaB1Rxx0iNv8vvDosNXCUR.jpg","backdropURLs":{"1280":"https://image.tmdb.org/t/p/w1280/xKmTsbaB1Rxx0iNv8vvDosNXCUR.jpg","300":"https://image.tmdb.org/t/p/w300/xKmTsbaB1Rxx0iNv8vvDosNXCUR.jpg","780":"https://image.tmdb.org/t/p/w780/xKmTsbaB1Rxx0iNv8vvDosNXCUR.jpg","original":"https://image.tmdb.org/t/p/original/xKmTsbaB1Rxx0iNv8vvDosNXCUR.jpg"},"originalTitle":"#GeorgeWashington","genres":[99,6],"countries":["US"],"year":2017,"runtime":12,"cast":[],"significants":[],"title":"#GeorgeWashington","overview":"Meet the Father of Our Country in a whole new way as historians analyze George Washington's voluminous correspondence.  Setting out his contacts and letters in the modern framework of a Facebook profile gives Washington's busy and productive career a new transparency.","tagline":"","video":"","posterPath":"/nE4w1cSFCBqdjWWNA01yxCkjRk0.jpg","posterURLs":{"154":"https

## Merging Datasets

In [ ]:
netflix = pd.read_csv('Dataset Netflix.csv', error_bad_lines=False, sep=';')
prime = pd.read_csv('Dataset Prime.csv', error_bad_lines=False, sep=';')
disney = pd.read_csv('Dataset Disney.csv', error_bad_lines=False, sep=';')
hbo = pd.read_csv('Dataset HBO.csv', error_bad_lines=False, sep=';')

In [ ]:
full_dataset = pd.concat([netflix, prime, disney, hbo], ignore_index=True)

In [ ]:
full_dataset

In [ ]:
NA_countries = []
SA_countries = []
EU_countries = []
AS_countries = []
AF_countries = []
OT_countries = []

In [ ]:
def convert_country_to_continent (country_column):
    total_rows = country_column.count
    continent_column = pd.DataFrame(columns=['continent'])
    for i in range(total_rows):
        entree = country_column.iloc[i]
        if entree in NA_countries:
            continent_column.iloc[i] = ['North America']
        elif entree in SA_countries:
            continent_column.iloc[i] = ['South America']
        elif entree in EU_countries:
            continent_column.iloc[i] = ['Europe']
        elif entree in AS_countries:
            continent_column.iloc[i] = ['Asia']
        elif entree in AF_countries:
            continent_column.iloc[i] = ['Africa']
        elif entree in OT_countries:
            continent_column.iloc[i] = ['Others']

In [ ]:
for i in range(full_dataset['imdbID'].count()):
    if type(full_dataset.iloc[i]['countries']) != float:
        full_dataset.iloc[i]['countries'] = full_dataset.iloc[i]['countries'][:4]
    else:
        full_dataset.iloc[i]['countries'] = '[]'

In [ ]:
def Convert(string):
    if type(string) == string && len(list(string.split("'"))) :
        li = list(string.split("'"))
    else:
        li = ''
    return li

In [ ]:
full_dataset['countries'].map(Convert)

In [ ]:
new_countries = pd.DataFrame(columns=['countries'])

for i in range(full_dataset['imdbID'].count()):
    if type(full_dataset.iloc[i]['countries']) != float:
        print(new_countries.shape)
        new_countries.iloc[i]['countries'] = Convert(full_dataset.iloc[i]['countries'])[1]

new_countries

In [ ]:
full_dataset.to_csv('nodes.csv')